# Generating dataset for ML classifier

## Imports and Config

In [2]:
import yfinance as yf
import pandas as pd
import numpy as np
from ta.momentum import RSIIndicator
from ta.volatility import AverageTrueRange
import matplotlib.pyplot as plt
import seaborn as sns
import requests
from bs4 import BeautifulSoup

sns.set(style="whitegrid")

# ─── CONFIG ────────────────────────────────────
start_date = "2004-01-01"
end_date   = "2025-12-31"
horizon    = 1    # days ahead for the target


In [4]:
def get_sp500_tickers():
    """
    Scrape the current list of S&P 500 tickers from Wikipedia.
    Uses requests + BeautifulSoup to avoid HTTP 403 errors.
    """
    url = "https://en.wikipedia.org/wiki/List_of_S%26P_500_companies"

    # Add browser headers to bypass bot protection
    headers = {
        "User-Agent": (
            "Mozilla/5.0 (Windows NT 10.0; Win64; x64) "
            "AppleWebKit/537.36 (KHTML, like Gecko) "
            "Chrome/120.0 Safari/537.36"
        )
    }

    response = requests.get(url, headers=headers)
    response.raise_for_status()   # will show any real connection error

    # Parse the HTML with BeautifulSoup
    soup = BeautifulSoup(response.text, "lxml")
    table = soup.find("table", {"id": "constituents"})

    if table is None:
        raise ValueError("Could not find the constituents table on the Wikipedia page.")

    # Extract the table using pandas
    df = pd.read_html(str(table))[0]

    tickers = [t.replace(".", "-") for t in df["Symbol"].astype(str).tolist()]
    return tickers

# Usage
sp500_tickers = get_sp500_tickers()
print(f"✅ Fetched {len(sp500_tickers)} tickers, e.g.: {sp500_tickers[:10]}")


✅ Fetched 503 tickers, e.g.: ['MMM', 'AOS', 'ABT', 'ABBV', 'ACN', 'ADBE', 'AMD', 'AES', 'AFL', 'A']


C:\Users\rahul\AppData\Local\Temp\ipykernel_39944\262162819.py:28: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  df = pd.read_html(str(table))[0]


In [5]:
tickers = get_sp500_tickers()

C:\Users\rahul\AppData\Local\Temp\ipykernel_39944\262162819.py:28: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  df = pd.read_html(str(table))[0]


## Step 1: Fetch Price Data for All Tickers

In [6]:
# ── Step 1: Fetch price data for the entire S&P 500 ───────────
print("Fetching S&P 500 ticker list…")
tickers = get_sp500_tickers()
print(f"Got {len(tickers)} tickers. Date range: {start_date} → {end_date}")

# batch size to avoid timeouts / throttling
batch_size = 50
chunks = [tickers[i:i+batch_size] for i in range(0, len(tickers), batch_size)]

frames = []
for chunk in chunks:
    print(f"  Downloading tickers {chunk[0]} … {chunk[-1]} …")
    df_chunk = yf.download(
        chunk,
        start=start_date,
        end=end_date,
        group_by="ticker",
        auto_adjust=False,
        threads=True
    )
    frames.append(df_chunk)

# concatenate all batches side-by-side
raw = pd.concat(frames, axis=1)

# drop rows where *all* tickers missed (e.g. holidays)
raw.dropna(how="all", inplace=True)

print("✅ Price download complete. Raw shape:", raw.shape)
display(raw.head())


Fetching S&P 500 ticker list…


C:\Users\rahul\AppData\Local\Temp\ipykernel_39944\262162819.py:28: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  df = pd.read_html(str(table))[0]


Got 503 tickers. Date range: 2004-01-01 → 2025-12-31


[*********************100%***********************]  50 of 50 completed


[*********************100%***********************]  50 of 50 completed


...

[*********************100%***********************]  50 of 50 completed


[*********************100%***********************]  50 of 50 completed


[*********************100%***********************]  3 of 3 completed


✅ Price download complete. Raw shape: (5495, 3018)


Ticker     GOOG                                        AJG             \
Price      Open High Low Close Adj Close Volume       Open       High   
Date                                                                    
2004-01-02  NaN  NaN NaN   NaN       NaN    NaN  32.400002  32.650002   
2004-01-05  NaN  NaN NaN   NaN       NaN    NaN  31.969999  32.810001   
2004-01-06  NaN  NaN NaN   NaN       NaN    NaN  32.400002  32.419998   
2004-01-07  NaN  NaN NaN   NaN       NaN    NaN  31.620001  31.740000   
2004-01-08  NaN  NaN NaN   NaN       NaN    NaN  31.799999  32.209999   

Ticker                            ...        ZBH                        \
Price             Low      Close  ...        Low      Close  Adj Close   
Date                              ...                                    
2004-01-02  31.740000  31.950001  ...  67.864075  67.961166  60.486706   
2004-01-05  31.969999  32.310001  ...  66.990288  67.349518  59.942310   
2004-01-06  31.450001  31.620001  ...  66.252426  67.019417  59.648548   
2004-01-07  31.379999  31.709999  ...  66.912621  67.766991  60.313881   
2004-01-08  31.650000  32.080002  ...  67.592232  69.077667  61.480404   

Ticker                    ZBRA                                              \
Price        Volume       Open       High        Low      Close  Adj Close   
Date                                                                         
2004-01-02   688143  44.126667  44.186668  43.406666  43.586666  43.586666   
2004-01-05  1223846  43.486668  44.180000  43.393333  43.666668  43.666668   
2004-01-06  1202216  43.566666  43.886665  43.366669  43.766666  43.766666   
2004-01-07   866127  43.633331  44.653332  43.500000  44.500000  44.500000   
2004-01-08  1114151  44.939999  45.246666  44.133331  44.599998  44.599998   

Ticker              
Price       Volume  
Date                
2004-01-02  339900  
2004-01-05  640950  
2004-01-06  311700  
2004-01-07  495150  
2004-01-08  290850  

[5 rows x 3018 columns]

## Step 2: Build Unified Event Dates Table (Earnings Only)

In [ ]:
tickers

['MMM',
 'AOS',
 'ABT',
 'ABBV',
 'ACN',
 'ADBE',
 'AMD',
 'AES',
 'AFL',
 'A',
 'APD',
 'ABNB',
 'AKAM',
 'ALB',
 'ARE',
 'ALGN',
 'ALLE',
 'LNT',
 'ALL',
 'GOOGL',
 'GOOG',
 'MO',
 'AMZN',
 'AMCR',
 'AEE',
 'AEP',
 'AXP',
 'AIG',
 'AMT',
 'AWK',
 'AMP',
 'AME',
 'AMGN',
 'APH',
 'ADI',
 'AON',
 'APA',
 'APO',
 'AAPL',
 'AMAT',
 'APP',
 'APTV',
 'ACGL',
 'ADM',
 'ANET',
 'AJG',
 'AIZ',
 'T',
 'ATO',
 'ADSK',
 'ADP',
 'AZO',
 'AVB',
 'AVY',
 'AXON',
 'BKR',
 'BALL',
 'BAC',
 'BAX',
 'BDX',
 'BRK-B',
 'BBY',
 'TECH',
 'BIIB',
 'BLK',
 'BX',
 'XYZ',
 'BK',
 'BA',
 'BKNG',
 'BSX',
 'BMY',
 'AVGO',
 'BR',
 'BRO',
 'BF-B',
 'BLDR',
 'BG',
 'BXP',
 'CHRW',
 'CDNS',
 'CPT',
 'CPB',
 'COF',
 'CAH',
 'CCL',
 'CARR',
 'CAT',
 'CBOE',
 'CBRE',
 'CDW',
 'COR',
 'CNC',
 'CNP',
 'CF',
 'CRL',
 'SCHW',
 'CHTR',
 'CVX',
 'CMG',
 'CB',
 'CHD',
 'CI',
 'CINF',
 'CTAS',
 'CSCO',
 'C',
 'CFG',
 'CLX',
 'CME',
 'CMS',
 'KO',
 'CTSH',
 'COIN',
 'CL',
 'CMCSA',
 'CAG',
 'COP',
 'ED',
 'STZ',
 'CEG',
 'COO',


In [9]:
bad_tickers = []
events = []

for t in tickers:
    try:
        print(f"Gathering earnings for {t}…", end=" ")

        ticker_obj = yf.Ticker(t)
        ed = ticker_obj.earnings_dates

        # skip if None or empty DataFrame
        if ed is None or ed.empty:
            print("⚠️  No data — skipping")
            bad_tickers.append(t)
            continue

        ed = ed.reset_index()
        ed.columns = ["Date", "Estimate", "Reported", "Surprise_%"]

        # normalize dates
        ed["Date"] = ed["Date"].dt.tz_localize(None).dt.normalize()
        ed["Ticker"] = t
        events.append(ed[["Date", "Ticker"]])
        print("✅ done")

    except Exception as e:
        print(f"❌ Error: {e}")
        bad_tickers.append(t)
        continue

# Combine all successful results
if events:
    earnings_dates = pd.concat(events, ignore_index=True)
    earnings_dates = earnings_dates[
        (earnings_dates["Date"] >= pd.to_datetime(start_date))
        & (earnings_dates["Date"] <= pd.to_datetime(end_date))
    ].sort_values(["Date", "Ticker"]).reset_index(drop=True)
else:
    earnings_dates = pd.DataFrame(columns=["Date", "Ticker"])

print(f"\n✅ Total valid earnings events: {len(earnings_dates)}")
print(f"🚫 Skipped {len(bad_tickers)} tickers with no data: {bad_tickers[:10]}…")


Gathering earnings for MMM… ✅ done
Gathering earnings for AOS… ✅ done
Gathering earnings for ABT… ✅ done
Gathering earnings for ABBV… ✅ done
Gathering earnings for ACN… ✅ done
Gathering earnings for ADBE… ✅ done
Gathering earnings for AMD… ✅ done
Gathering earnings for AES… ✅ done
Gathering earnings for AFL… ✅ done
Gathering earnings for A… ✅ done
Gathering earnings for APD… ✅ done
Gathering earnings for ABNB… ✅ done
Gathering earnings for AKAM… ✅ done
Gathering earnings for ALB… ✅ done
Gathering earnings for ARE… ✅ done
Gathering earnings for ALGN… ✅ done
Gathering earnings for ALLE… ✅ done
Gathering earnings for LNT… ✅ done
Gathering earnings for ALL… ✅ done
Gathering earnings for GOOGL… 

C:\Users\rahul\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\yfinance\base.py:761: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df['Earnings Date'] = pd.to_datetime(df['Event Start Date'])


✅ done
Gathering earnings for GOOG… ✅ done
Gathering earnings for MO… ✅ done
Gathering earnings for AMZN… ✅ done
Gathering earnings for AMCR… ✅ done
Gathering earnings for AEE… ✅ done
Gathering earnings for AEP… ✅ done
Gathering earnings for AXP… ✅ done
Gathering earnings for AIG… ✅ done
Gathering earnings for AMT… ✅ done
Gathering earnings for AWK… ✅ done
Gathering earnings for AMP… ✅ done
Gathering earnings for AME… ✅ done
Gathering earnings for AMGN… ✅ done
Gathering earnings for APH… ✅ done
Gathering earnings for ADI… ✅ done
Gathering earnings for AON… ✅ done
Gathering earnings for APA… ✅ done
Gathering earnings for APO… ✅ done
Gathering earnings for AAPL… ✅ done
Gathering earnings for AMAT… ✅ done
Gathering earnings for APP… ✅ done
Gathering earnings for APTV… ✅ done
Gathering earnings for ACGL… ✅ done
Gathering earnings for ADM… ✅ done
Gathering earnings for ANET… ✅ done
Gathering earnings for AJG… ✅ done
Gathering earnings for AIZ… ✅ done
Gathering earnings for T… ✅ done
Gatheri

FOX: $FOX: possibly delisted; no earnings dates found


✅ done
Gathering earnings for FOX… ⚠️  No data — skipping
Gathering earnings for BEN… ✅ done
Gathering earnings for FCX… ✅ done
Gathering earnings for GRMN… ✅ done
Gathering earnings for IT… ✅ done
Gathering earnings for GE… ✅ done
Gathering earnings for GEHC… ✅ done
Gathering earnings for GEV… ✅ done
Gathering earnings for GEN… ✅ done
Gathering earnings for GNRC… ✅ done
Gathering earnings for GD… ✅ done
Gathering earnings for GIS… ✅ done
Gathering earnings for GM… ✅ done
Gathering earnings for GPC… ✅ done
Gathering earnings for GILD… ✅ done
Gathering earnings for GPN… ✅ done
Gathering earnings for GL… ✅ done
Gathering earnings for GDDY… ✅ done
Gathering earnings for GS… ✅ done
Gathering earnings for HAL… ✅ done
Gathering earnings for HIG… ✅ done
Gathering earnings for HAS… ✅ done
Gathering earnings for HCA… ✅ done
Gathering earnings for DOC… ✅ done
Gathering earnings for HSIC… ✅ done
Gathering earnings for HSY… ✅ done
Gathering earnings for HPE… ✅ done
Gathering earnings for HLT… ✅ do

...

✅ done
Gathering earnings for NWS… ⚠️  No data — skipping
Gathering earnings for NEE… ✅ done
Gathering earnings for NKE… ✅ done
Gathering earnings for NI… ✅ done
Gathering earnings for NDSN… ✅ done
Gathering earnings for NSC… ✅ done
Gathering earnings for NTRS… ✅ done
Gathering earnings for NOC… ✅ done
Gathering earnings for NCLH… ✅ done
Gathering earnings for NRG… ✅ done
Gathering earnings for NUE… ✅ done
Gathering earnings for NVDA… ✅ done
Gathering earnings for NVR… ✅ done
Gathering earnings for NXPI… ✅ done
Gathering earnings for ORLY… ✅ done
Gathering earnings for OXY… ✅ done
Gathering earnings for ODFL… ✅ done
Gathering earnings for OMC… ✅ done
Gathering earnings for ON… ✅ done
Gathering earnings for OKE… ✅ done
Gathering earnings for ORCL… ✅ done
Gathering earnings for OTIS… ✅ done
Gathering earnings for PCAR… ✅ done
Gathering earnings for PKG… ✅ done
Gathering earnings for PLTR… ✅ done
Gathering earnings for PANW… ✅ done
Gathering earnings for PSKY… 

PSKY: $PSKY: possibly delisted; no earnings dates found


⚠️  No data — skipping
Gathering earnings for PH… ✅ done
Gathering earnings for PAYX… ✅ done
Gathering earnings for PAYC… ✅ done
Gathering earnings for PYPL… ✅ done
Gathering earnings for PNR… ✅ done
Gathering earnings for PEP… ✅ done
Gathering earnings for PFE… ✅ done
Gathering earnings for PCG… ✅ done
Gathering earnings for PM… ✅ done
Gathering earnings for PSX… ✅ done
Gathering earnings for PNW… ✅ done
Gathering earnings for PNC… ✅ done
Gathering earnings for POOL… ✅ done
Gathering earnings for PPG… ✅ done
Gathering earnings for PPL… ✅ done
Gathering earnings for PFG… ✅ done
Gathering earnings for PG… ✅ done
Gathering earnings for PGR… ✅ done
Gathering earnings for PLD… ✅ done
Gathering earnings for PRU… ✅ done
Gathering earnings for PEG… ✅ done
Gathering earnings for PTC… ✅ done
Gathering earnings for PSA… ✅ done
Gathering earnings for PHM… ✅ done
Gathering earnings for PWR… ✅ done
Gathering earnings for QCOM… ✅ done
Gathering earnings for DGX… ✅ done
Gathering earnings for RL… ✅ d

SOLS: $SOLS: possibly delisted; no earnings dates found


⚠️  No data — skipping
Gathering earnings for SOLV… ✅ done
Gathering earnings for SO… ✅ done
Gathering earnings for LUV… ✅ done
Gathering earnings for SWK… ✅ done
Gathering earnings for SBUX… ✅ done
Gathering earnings for STT… ✅ done
Gathering earnings for STLD… ✅ done
Gathering earnings for STE… ✅ done
Gathering earnings for SYK… ✅ done
Gathering earnings for SMCI… ✅ done
Gathering earnings for SYF… ✅ done
Gathering earnings for SNPS… ✅ done
Gathering earnings for SYY… ✅ done
Gathering earnings for TMUS… ✅ done
Gathering earnings for TROW… ✅ done
Gathering earnings for TTWO… ✅ done
Gathering earnings for TPR… ✅ done
Gathering earnings for TRGP… ✅ done
Gathering earnings for TGT… ✅ done
Gathering earnings for TEL… ✅ done
Gathering earnings for TDY… ✅ done
Gathering earnings for TER… ✅ done
Gathering earnings for TSLA… ✅ done
Gathering earnings for TXN… ✅ done
Gathering earnings for TPL… ✅ done
Gathering earnings for TXT… ✅ done
Gathering earnings for TMO… ✅ done
Gathering earnings for 

In [10]:
# Step 2 — Build unified earnings_dates with Surprise_% included
events = []

# Filter tickers to exclude those with no data (collected earlier)
good_tickers = [t for t in tickers if t not in bad_tickers]
print(f"Processing {len(good_tickers)} valid tickers (skipping {len(bad_tickers)} bad ones)")

for t in good_tickers:
    try:
        ed = yf.Ticker(t).earnings_dates
        if ed is None or ed.empty:
            print(f"⚠️  No earnings data for {t} — skipping")
            bad_tickers.append(t)
            continue

        ed = ed.reset_index()
        ed.columns = ["Date", "Earnings_Estimate", "Reported_Earnings", "Surprise_%"]
        ed["Date"] = ed["Date"].dt.tz_localize(None).dt.normalize()
        ed["Ticker"] = t
        events.append(ed[["Date", "Ticker", "Surprise_%"]])

    except Exception as e:
        print(f"❌ {t}: {e}")
        bad_tickers.append(t)
        continue

earnings_dates = (
    pd.concat(events, ignore_index=True)
      .query("@start_date <= Date <= @end_date")
      .sort_values(["Date", "Ticker"])
      .reset_index(drop=True)
)

print(f"✅ Total earnings events (with Surprise_%): {len(earnings_dates)}")
display(earnings_dates.head())

Processing 499 valid tickers (skipping 4 bad ones)


C:\Users\rahul\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\yfinance\base.py:761: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df['Earnings Date'] = pd.to_datetime(df['Event Start Date'])


✅ Total earnings events (with Surprise_%): 5901


,Date,Ticker,Surprise_%
0,2021-11-01,L,NaN
1,2022-02-07,L,NaN
2,2022-05-03,SMCI,38.70
3,2022-06-30,STZ,5.89
4,2022-07-12,PEP,7.11


## Step 3: Feature Engineering Per Ticker

In [ ]:
feature_list = []

for t in tickers:
    print(f"  Engineering features for {t}…")
    df_t = raw[t].copy()  # slice out ticker
    
    # 1) flatten MultiIndex
    if isinstance(df_t.columns, pd.MultiIndex):
        df_t.columns = df_t.columns.get_level_values(0)
    # 2) drop stray Price
    df_t.drop(columns=[c for c in ["Price"] if c in df_t], inplace=True)
    
    # 3) coerce core series
    df_t["Close"]  = pd.to_numeric(df_t["Close"],  errors="coerce")
    df_t["Volume"] = pd.to_numeric(df_t["Volume"], errors="coerce")
    df_t.dropna(subset=["Close","Volume"], inplace=True)
    
    # 4) basic features
    df_t["Return"]      = df_t["Close"].pct_change()
    df_t["Volatility"]  = df_t["Return"].rolling(5).std()
    df_t["RSI"]         = RSIIndicator(close=df_t["Close"], window=14).rsi()
    df_t["MA5"]         = df_t["Close"].rolling(5).mean()
    df_t["MA10"]        = df_t["Close"].rolling(10).mean()
    df_t["MA_ratio"]    = df_t["MA5"] / df_t["MA10"] - 1
    df_t["Volume_Avg20"]= df_t["Volume"].rolling(20).mean()
    df_t["Volume_Spike"]= df_t["Volume"] / df_t["Volume_Avg20"] - 1
    
    # 5) new predictive features
    df_t["Momentum3"]   = df_t["Close"].pct_change(3)
    atr = AverageTrueRange(high=df_t["High"], low=df_t["Low"], 
                           close=df_t["Close"], window=14)
    df_t["ATR14"]       = atr.average_true_range()
    df_t["DayOfWeek"]   = df_t.index.dayofweek
    df_t["Month"]       = df_t.index.month

    # ——— HERE: merge in Surprise_% from your earnings_dates ———
    # slice out only this ticker’s surprises
    ed_t = (
        earnings_dates
        .loc[earnings_dates["Ticker"] == t, ["Date","Surprise_%"]]
        .set_index("Date")
    )
    # align on the same dates, left‐join so non-event days get NaN
    df_t = df_t.join(ed_t, how="left")
    # fill non‐events with zero surprise
    df_t["Surprise_%"] = df_t["Surprise_%"].fillna(0)
    
    # 6) drop NaNs from rolling/pct_change (but keep Surprise_% zeros)
    df_t.dropna(subset=[
        "Return","Volatility","RSI","MA5","MA10","MA_ratio",
        "Volume_Avg20","Volume_Spike","Momentum3","ATR14"
    ], inplace=True)
    
    # 7) select your expanded feature set
    keep = [
        "Close","Volume",
        "Return","Volatility","RSI",
        "MA5","MA10","MA_ratio",
        "Volume_Avg20","Volume_Spike",
        "Momentum3","ATR14",
        "DayOfWeek","Month",
        "Surprise_%"
    ]
    feats = df_t[keep].copy()
    feats["Ticker"] = t
    feature_list.append(feats)

# concatenate all tickers
features_df = pd.concat(feature_list)
features_df.index.name = "Date"
features_df = features_df.sort_index()

print("Features shape (all tickers):", features_df.shape)
display(features_df.head())

## Step 4: Create Event-Only Labels

In [54]:
# Step 4: Label creation (target variable) for multi‐ticker MultiIndex

def create_labels(event_dates, price_df, horizon=3):
    """
    event_dates: DataFrame with ['Date','Ticker'] columns of pd.Timestamps
    price_df:    DataFrame with a MultiIndex (Date, Ticker) and at least a 'Close' column
    horizon:     how many trading days ahead to look
    """
    labels = []
    # 1) pre‐shift the Close series within each ticker
    future_close = price_df['Close'].groupby(level='Ticker').shift(-horizon)
    
    for _, ev in event_dates.iterrows():
        dt, tkr = ev['Date'], ev['Ticker']
        key = (dt, tkr)
        # 2) skip if that (Date, Ticker) combo isn't in your features
        if key not in price_df.index:
            continue
        
        past = price_df.at[key, 'Close']
        fut  = future_close.at[key]
        # 3) skip if we ran off the end
        if pd.isna(fut):
            continue
        
        ret = (fut - past) / past
        labels.append({
          'Date':   dt,
          'Ticker': tkr,
          'Target': int(ret > 0)
        })
    
    return pd.DataFrame(labels)


# — how to call it —
# make sure features_df is a MultiIndexed DF: index names must be ['Date','Ticker']
features_df = features_df.reset_index().set_index(['Date','Ticker'])

labels_df = create_labels(earnings_dates, features_df, horizon=horizon)
print(f"Labeled {len(labels_df)} events:")
display(labels_df)

Labeled 2963 events:


,Date,Ticker,Target
0,2022-07-28,KIM,1
1,2022-08-01,SPG,0
2,2022-08-04,FRT,1
3,2022-08-17,AMCR,0
4,2022-10-27,AOS,1
...,...,...,...
2958,2024-12-19,KMX,1
2959,2024-12-19,LW,0
2960,2024-12-19,NKE,0
2961,2024-12-19,PAYX,1


## Step 5: Merge features & labels for multi-ticker dataset

In [55]:
# 1) Ensure the feature and label DataFrames share the same MultiIndex
#    (Date,Ticker) before joining:

# features_df should already be indexed by (Date,Ticker)
# if not, do it explicitly:
features_df = features_df.reset_index().set_index(['Date','Ticker'])

# labels_df just needs to have the same index
labels_df = labels_df.set_index(['Date','Ticker'])

# 2) Join on that MultiIndex, pulling in only the 'Target' column from labels_df
final_df = features_df.join(
    labels_df[['Target']],
    how='inner'
).reset_index()

# 3) Inspect & save
print("Final dataset shape:", final_df.shape)
display(final_df.head())

final_df.to_csv("multi_ticker_earnings_dataset.csv", index=False)
print("✅ Saved to multi_ticker_earnings_dataset.csv")

Final dataset shape: (2963, 17)


,Date,Ticker,Close,Volume,Return,Volatility,RSI,MA5,MA10,MA_ratio,Volume_Avg20,Volume_Spike,Momentum3,ATR14,DayOfWeek,Month,Target
0,2022-07-28,KIM,21.860001,4028800.0,0.016744,0.013179,62.512911,21.498,21.180000,0.015014,3674420.0,0.096445,0.016272,0.522246,3,7,1
1,2022-08-01,SPG,108.629997,1650100.0,-0.000092,0.017185,65.657759,106.286,104.909999,0.013116,1648115.0,0.001204,0.043315,2.614542,0,8,0
2,2022-08-04,FRT,105.040001,797300.0,0.008352,0.009903,59.129102,104.944,103.873000,0.010311,644255.0,0.237553,-0.007090,2.419999,3,8,1
3,2022-08-17,AMCR,13.010000,8882400.0,-0.021068,0.017779,56.644113,12.954,12.713000,0.018957,9221750.0,-0.036799,0.015613,0.295485,2,8,0
4,2022-10-27,AOS,51.900002,1332100.0,-0.000962,0.014431,52.561661,51.376,50.882000,0.009709,1311860.0,0.015428,0.021654,1.601343,3,10,1


✅ Saved to multi_ticker_earnings_dataset.csv
